In [5]:
import pandas as pd
import numpy as np
import os

In [6]:
data_folder = "../data"

# Get list of data directories
data_dirs = os.listdir(data_folder)
data_dirs = [d for d in data_dirs if os.path.isdir(os.path.join(data_folder, d))]
data_dirs.sort()
data_dirs

['2025-05-23--12h-20m-47s',
 '2025-05-23--12h-20m-57s',
 '2025-05-23--12h-21m-01s',
 '2025-05-23--12h-21m-05s',
 '2025-05-23--12h-21m-09s',
 '2025-05-23--12h-21m-13s',
 '2025-05-23--12h-21m-17s',
 '2025-05-23--12h-21m-20s',
 '2025-05-23--12h-21m-24s',
 '2025-05-23--12h-21m-28s',
 '2025-05-23--12h-21m-32s',
 '2025-05-23--12h-21m-36s',
 '2025-05-23--12h-21m-40s',
 '2025-05-23--12h-21m-44s',
 '2025-05-23--12h-21m-48s',
 '2025-05-23--12h-21m-52s',
 '2025-05-23--12h-21m-56s',
 '2025-05-23--12h-22m-00s',
 '2025-05-23--12h-22m-04s',
 '2025-05-23--12h-22m-08s',
 '2025-05-23--12h-22m-12s',
 '2025-05-23--12h-22m-16s',
 '2025-05-23--12h-22m-20s',
 '2025-05-23--12h-22m-24s',
 '2025-05-23--12h-22m-28s',
 '2025-05-23--12h-22m-32s',
 '2025-05-23--12h-22m-36s',
 '2025-05-23--12h-22m-40s',
 '2025-05-23--12h-22m-45s',
 '2025-05-23--12h-22m-49s',
 '2025-05-23--12h-22m-53s',
 '2025-05-23--12h-22m-57s',
 '2025-05-23--12h-23m-01s',
 '2025-05-23--12h-23m-05s',
 '2025-05-23--12h-23m-09s',
 '2025-05-23--12h-23

In [7]:
new_data_dirs = data_dirs.copy()
# Remove directories that already have theta_delta_phi_estimates.csv
for d in data_dirs:
    if os.path.exists(os.path.join(data_folder, d, "theta_delta_phi_estimates.csv")):
        new_data_dirs.remove(d)

new_data_dirs

[]

In [8]:
def load_coincidences(data_dir):
    coincidences = pd.read_csv(os.path.join(data_folder, data_dir, "coincidences.csv"))
    coincidences["data_dir"] = data_dir
    return coincidences

coincidences_df = pd.concat([load_coincidences(d) for d in new_data_dirs], ignore_index=True)
coincidences_df

ValueError: No objects to concatenate

Double bunched events are only resolved half of the time. Therefore, we now throw away half of all non-double-bunched events to recover the expected statistics.

In [ ]:
coincidences_df["scaled_coincidences"] = coincidences_df["coincidences"].astype(float)

coincidences_df.loc[coincidences_df["estimation_label"] == "SB", "scaled_coincidences"] = coincidences_df.loc[coincidences_df["estimation_label"] == "SB", "coincidences"] * 0.5
coincidences_df.loc[coincidences_df["estimation_label"] == "C", "scaled_coincidences"] = coincidences_df.loc[coincidences_df["estimation_label"] == "C", "coincidences"] * 0.5

coincidences_df

,detector_a_name,detector_b_name,arm_a,arm_b,color_a,color_b,delay_a,delay_b,estimation_label,coincidences,data_dir,scaled_coincidences
0,9,12,TT,TT,white,blue,1.016000e-08,1.172000e-08,DB_H,1194,2025-05-23--12h-23m-01s,1194.0
1,9,11,TT,TR,white,white,1.016000e-08,1.172000e-08,SB,12,2025-05-23--12h-23m-01s,6.0
2,9,10,TT,TR,white,blue,1.016000e-08,1.016000e-08,SB,12,2025-05-23--12h-23m-01s,6.0
3,11,12,TR,TT,white,blue,1.172000e-08,1.172000e-08,SB,15,2025-05-23--12h-23m-01s,7.5
4,10,12,TR,TT,blue,blue,1.016000e-08,1.172000e-08,SB,19,2025-05-23--12h-23m-01s,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...
779,2,12,RR,TT,blue,blue,3.120000e-09,1.172000e-08,C,0,2025-05-23--12h-24m-49s,0.0
780,2,11,RR,TR,blue,white,3.120000e-09,1.172000e-08,C,0,2025-05-23--12h-24m-49s,0.0
781,2,10,RR,TR,blue,blue,3.120000e-09,1.016000e-08,C,160,2025-05-23--12h-24m-49s,80.0
782,2,4,RR,RT,blue,blue,3.120000e-09,3.910000e-09,SB,0,2025-05-23--12h-24m-49s,0.0


In [ ]:
# Sum up by estimation label
df_sum = (
    coincidences_df.groupby(["data_dir", "estimation_label"])
    .agg({"scaled_coincidences": "sum"})
    .reset_index()
)
df_sum

,data_dir,estimation_label,scaled_coincidences
0,2025-05-23--12h-23m-01s,C,6559.0
1,2025-05-23--12h-23m-01s,DB_H,2908.0
2,2025-05-23--12h-23m-01s,DB_V,3974.0
3,2025-05-23--12h-23m-01s,SB,38.5
4,2025-05-23--12h-23m-05s,C,6506.5
...,...,...,...
107,2025-05-23--12h-24m-45s,SB,4.0
108,2025-05-23--12h-24m-49s,C,254.0
109,2025-05-23--12h-24m-49s,DB_H,0.0
110,2025-05-23--12h-24m-49s,DB_V,12878.0


In [ ]:
df_pivoted_sum = df_sum.pivot(index='data_dir', columns='estimation_label', values='scaled_coincidences')
df_pivoted_sum = df_pivoted_sum.reset_index()  # optional, to flatten the index
df_pivoted_sum.columns.name = None  # remove the name of the columns
df_pivoted_sum

,data_dir,C,DB_H,DB_V,SB
0,2025-05-23--12h-23m-01s,6559.0,2908.0,3974.0,38.5
1,2025-05-23--12h-23m-05s,6506.5,2586.0,4264.0,34.5
2,2025-05-23--12h-23m-09s,6430.0,2259.0,4803.0,29.0
3,2025-05-23--12h-23m-13s,6199.5,2013.0,5220.0,35.0
4,2025-05-23--12h-23m-17s,6072.5,1722.0,5613.0,37.0
5,2025-05-23--12h-23m-21s,5795.0,1437.0,5893.0,32.0
6,2025-05-23--12h-23m-25s,5548.0,1197.0,6483.0,33.0
7,2025-05-23--12h-23m-29s,5241.0,1008.0,6911.0,32.0
8,2025-05-23--12h-23m-33s,5000.0,887.0,7302.0,25.0
9,2025-05-23--12h-23m-37s,4641.5,718.0,7682.0,31.5


In [ ]:
df_pivoted_sum["N"] = df_pivoted_sum["SB"] + df_pivoted_sum["C"] + df_pivoted_sum["DB_H"] + df_pivoted_sum["DB_V"]
df_pivoted_sum

,data_dir,C,DB_H,DB_V,SB,N
0,2025-05-23--12h-23m-01s,6559.0,2908.0,3974.0,38.5,13479.5
1,2025-05-23--12h-23m-05s,6506.5,2586.0,4264.0,34.5,13391.0
2,2025-05-23--12h-23m-09s,6430.0,2259.0,4803.0,29.0,13521.0
3,2025-05-23--12h-23m-13s,6199.5,2013.0,5220.0,35.0,13467.5
4,2025-05-23--12h-23m-17s,6072.5,1722.0,5613.0,37.0,13444.5
5,2025-05-23--12h-23m-21s,5795.0,1437.0,5893.0,32.0,13157.0
6,2025-05-23--12h-23m-25s,5548.0,1197.0,6483.0,33.0,13261.0
7,2025-05-23--12h-23m-29s,5241.0,1008.0,6911.0,32.0,13192.0
8,2025-05-23--12h-23m-33s,5000.0,887.0,7302.0,25.0,13214.0
9,2025-05-23--12h-23m-37s,4641.5,718.0,7682.0,31.5,13073.0


In [ ]:
df_pivoted_sum["theta_estimate"] = np.arccos(
    (df_pivoted_sum["DB_H"] - df_pivoted_sum["DB_V"]) / df_pivoted_sum["N"]
)
df_pivoted_sum["delta_phi_estimate"] = np.arctan(
    np.sqrt(df_pivoted_sum["C"] / df_pivoted_sum["SB"])
)
df_pivoted_sum

,data_dir,C,DB_H,DB_V,SB,N,theta_estimate,delta_phi_estimate
0,2025-05-23--12h-23m-01s,6559.0,2908.0,3974.0,38.5,13479.5,1.649962,1.494331
1,2025-05-23--12h-23m-05s,6506.5,2586.0,4264.0,34.5,13391.0,1.696435,1.498107
2,2025-05-23--12h-23m-09s,6430.0,2259.0,4803.0,29.0,13521.0,1.760076,1.503740
3,2025-05-23--12h-23m-13s,6199.5,2013.0,5220.0,35.0,13467.5,1.811235,1.495800
4,2025-05-23--12h-23m-17s,6072.5,1722.0,5613.0,37.0,13444.5,1.864409,1.492896
5,2025-05-23--12h-23m-21s,5795.0,1437.0,5893.0,32.0,13157.0,1.916309,1.496622
6,2025-05-23--12h-23m-25s,5548.0,1197.0,6483.0,33.0,13261.0,1.980800,1.493825
7,2025-05-23--12h-23m-29s,5241.0,1008.0,6911.0,32.0,13192.0,2.034729,1.492816
8,2025-05-23--12h-23m-33s,5000.0,887.0,7302.0,25.0,13214.0,2.077697,1.500203
9,2025-05-23--12h-23m-37s,4641.5,718.0,7682.0,31.5,13073.0,2.132585,1.488601


In [ ]:
df_pivoted_sum["theta_estimate_degrees"] = np.degrees(df_pivoted_sum["theta_estimate"])
df_pivoted_sum["delta_phi_estimate_degrees"] = np.degrees(df_pivoted_sum["delta_phi_estimate"])
df_pivoted_sum

,data_dir,C,DB_H,DB_V,SB,N,theta_estimate,delta_phi_estimate,theta_estimate_degrees,delta_phi_estimate_degrees
0,2025-05-23--12h-23m-01s,6559.0,2908.0,3974.0,38.5,13479.5,1.649962,1.494331,94.535861,85.618865
1,2025-05-23--12h-23m-05s,6506.5,2586.0,4264.0,34.5,13391.0,1.696435,1.498107,97.198545,85.835215
2,2025-05-23--12h-23m-09s,6430.0,2259.0,4803.0,29.0,13521.0,1.760076,1.503740,100.844943,86.157937
3,2025-05-23--12h-23m-13s,6199.5,2013.0,5220.0,35.0,13467.5,1.811235,1.495800,103.776129,85.703023
4,2025-05-23--12h-23m-17s,6072.5,1722.0,5613.0,37.0,13444.5,1.864409,1.492896,106.822758,85.536660
5,2025-05-23--12h-23m-21s,5795.0,1437.0,5893.0,32.0,13157.0,1.916309,1.496622,109.796414,85.750152
6,2025-05-23--12h-23m-25s,5548.0,1197.0,6483.0,33.0,13261.0,1.980800,1.493825,113.491466,85.589859
7,2025-05-23--12h-23m-29s,5241.0,1008.0,6911.0,32.0,13192.0,2.034729,1.492816,116.581360,85.532043
8,2025-05-23--12h-23m-33s,5000.0,887.0,7302.0,25.0,13214.0,2.077697,1.500203,119.043267,85.955309
9,2025-05-23--12h-23m-37s,4641.5,718.0,7682.0,31.5,13073.0,2.132585,1.488601,122.188132,85.290561


In [ ]:
# For each row, save the results to a CSV file

for index, row in df_pivoted_sum.iterrows():
    data_dir = row["data_dir"]
    theta_estimate = row["theta_estimate_degrees"]
    delta_phi_estimate = row["delta_phi_estimate_degrees"]
    
    # Create a new DataFrame for the current row
    results = pd.DataFrame({
        "estimation_label": ["C", "SB", "DB_H", "DB_V"],
        "scaled_coincidences": [row["C"], row["SB"], row["DB_H"], row["DB_V"]]
    })
    
    # Save the results to a CSV file
    results.to_csv(os.path.join(data_folder, data_dir, "scaled_coincidences.csv"), index=False)
    
    # Save the theta and delta_phi estimates to a CSV file
    theta_delta_phi = pd.DataFrame({
        "theta_estimate": [theta_estimate],
        "delta_phi_estimate": [delta_phi_estimate]
    })
    theta_delta_phi.to_csv(os.path.join(data_folder, data_dir, "theta_delta_phi_estimates.csv"), index=False)